In [2]:
import sys
sys.path.append('../Data_Feature')
sys.path.append('../Datasets')
print(sys.path)
import numpy as np
import os
import seaborn as sns
from scipy.stats import pearsonr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB # or GaussianNB if your data is normalized and continuous
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, make_scorer, matthews_corrcoef
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import zipfile 
from bioservices import KEGG
from K_func import translate_ko_terms

['c:\\Users\\eliah\\Documents\\Master\\Eliah-Masters\\GO', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\eliah\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages', 'C:\\Users\\eliah\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32', 'C:\\Users\\eliah\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\eliah\\AppDa

In [18]:
# Reads and processes GO terms
os.chdir("../Datasets")
zip_file_path_GO = 'terms_GO.zip'
csv_file_name_GO = 'terms_GO.csv'
zip_file_path_traits = 'reducedDataset.zip'
csv_file_name_traits = 'reducedDataset.csv'

# Read and preprocess KO terms
with zipfile.ZipFile(zip_file_path_GO, 'r') as zip_ref:
    with zip_ref.open(csv_file_name_GO) as file:
        GOs = pd.read_csv(file, index_col=0)


GOs['value'] = 1
X = GOs.pivot_table(index='key', columns='GO', values='value', fill_value=0)
#print(X.head())


# Read and preprocess traits data
with zipfile.ZipFile(zip_file_path_traits, 'r') as zip_ref:
    with zip_ref.open(csv_file_name_traits) as file:
        traits = pd.read_csv(file, sep=';')
traits['oxygen'] = traits['oxygen'].str.lower()

print(traits["oxygen"])


0       facultative
1         anaerobic
2           aerobic
3           aerobic
4         anaerobic
           ...     
3404        aerobic
3405    facultative
3406        aerobic
3407    facultative
3408        aerobic
Name: oxygen, Length: 3409, dtype: object


In [4]:
# Processes data for traits.
traits = pd.read_csv('reducedDataset.csv', sep=';')
# The missing values for the oxygen column are removed and then the keys are grouped together
y = traits.dropna(subset=['oxygen']).groupby('key').agg({'oxygen': lambda x: x.value_counts().index[0]})

In [5]:
# Finds all the common keys between X and Y due to descrepancies adter the removal of empty oxygen values
common_keys = X.index.intersection(y.index)

In [6]:
# Align X and Y based on common keys
X_aligned = X.loc[common_keys]
Y = y.loc[common_keys].values.ravel()
# Ensures X and Y are aligned
assert X_aligned.shape[0] == len(Y), "X and Y are not aligned"

In [76]:
clf = svm.SVC()
clf.fit(X_aligned,Y)

In [7]:
print("Shape of X:", X_aligned.shape)
print("Shape of Y:", Y.shape)


Shape of X: (3255, 14002)
Shape of Y: (3255,)
